In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import CCA
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge

In [2]:
dat = pd.read_csv("baseball_data.csv")

In [3]:
dat = dat.drop(columns = ['id', 'yearid','teamid','lgid','namefirst','namelast'])
dat = dat[dat['salary'] > 0]

In [4]:
positions = np.unique(dat['pos'])
#for pos in positions:
temp_array = np.array(dat['pos'])
for position in positions:
    indicator = np.zeros(len(dat['pos']))
    for j in range(len(temp_array)):
        if temp_array[j] == position:
            indicator[j] = 1
    #indicator = pd.Series(indicator, name = position)
#     print(indicator)
#     print(len(temp_array))
    dat[position] = indicator

dat = dat.drop(columns = ['pos', 'g_batting'])

### PCA Handling

In [5]:
y = np.array(dat['salary'])
X = np.array(dat.drop(columns = ['salary']))
#preds = np.exp(reg.predict(new_X))
# print(pca.singular_values_)

In [6]:
# G.x, GS, Runs, AB, O1, OSS, O2, O3, 1B, C, years
imp_POS = {'G.x':1000, 'GS':800, 'Runs':600, 'AB':500, 'O1':300, 'OSS':300, 'O2':250, 'O3':220, '1B':210, 'C':180, 'years':-10}
coefs = np.zeros(51)
for i in range(51):
    if dat.columns[i] in imp_POS.keys():
        coefs[i] = imp_POS[dat.columns[i]]
    else:
        coefs[i] = np.random.uniform(0,0.6)*100
salary_pred = X @ coefs
new_y = salary_pred + np.random.normal(0, 30000, len(salary_pred))
reg = LinearRegression().fit(X, new_y)
np.mean((reg.predict(X) - new_y)**2) ** 0.5


27780.73026598597

In [7]:
pca = PCA(n_components = 6)
pca.fit(X)
pca_X = pca.transform(X)
reg = LinearRegression().fit(pca_X, new_y)
np.mean((reg.predict(pca_X) - new_y)**2) ** 0.5

30192.08975094813

In [9]:
u, s, vh = np.linalg.svd(X)
s = np.roll(s, 1)

array([8.61523676e-01, 7.96968371e+04, 3.04661368e+04, 5.13400528e+03,
       3.10634911e+03, 2.12006613e+03, 1.58394108e+03, 1.40208290e+03,
       1.32234874e+03, 7.01703298e+02, 4.13668573e+02, 3.88731458e+02,
       3.04616290e+02, 2.57976827e+02, 2.42434178e+02, 1.86975673e+02,
       1.62464347e+02, 1.27057154e+02, 1.21365287e+02, 1.02907319e+02,
       8.48115519e+01, 6.55383320e+01, 5.90653887e+01, 5.66632117e+01,
       5.50352424e+01, 5.22848798e+01, 4.21634098e+01, 3.69654414e+01,
       3.48854106e+01, 3.19831742e+01, 3.07576985e+01, 2.76895538e+01,
       7.00272113e+00, 5.91226461e+00, 5.13141890e+00, 5.04184341e+00,
       4.83542712e+00, 4.38741686e+00, 4.18466573e+00, 3.93846248e+00,
       3.46302504e+00, 3.43286606e+00, 3.30412200e+00, 2.81136348e+00,
       2.52911409e+00, 2.35553242e+00, 2.04027404e+00, 1.89588664e+00,
       1.30458871e+00, 9.96752780e-01, 9.60071552e-01])

In [10]:
np.array([np.zeros(51)] * (421-51)).shape

(370, 51)

In [11]:
s = np.vstack([np.diag(s), np.array([np.zeros(51)] * (421-51))])
s.shape

(421, 51)

In [12]:
new_X = u @ s @ vh
new_X

array([[-3.46416226e+02, -3.26068727e+02, -8.90536912e+03, ...,
         2.43703151e+00, -2.29710104e-01, -1.87783579e-01],
       [ 2.23690737e+02,  2.15419777e+02,  5.81968022e+03, ...,
        -4.94899159e-01,  9.44307385e-02,  2.06512434e-01],
       [-7.74364712e+00, -7.84125042e+00, -2.11587590e+02, ...,
        -2.60671770e-01,  2.60314207e-02, -4.19995410e-02],
       ...,
       [-2.33600560e+02, -2.11725402e+02, -5.65899889e+03, ...,
        -8.86354001e-01, -9.20538336e-02, -3.61037550e-01],
       [ 5.94228343e+01,  7.13625131e+01,  1.89915592e+03, ...,
         3.95398072e-01,  3.65767539e-02,  3.88382833e-01],
       [ 1.59553786e+02,  1.39273307e+02,  3.70344480e+03, ...,
        -3.39787780e-01,  6.62360059e-02,  1.45600365e-01]])

In [13]:
pca = PCA(n_components=6)
pca.fit(new_X)
pca_X = pca.transform(new_X)
reg = LinearRegression().fit(pca_X, new_y)
np.mean((reg.predict(pca_X) - new_y)**2) ** 0.5

51615.023869393175

### CCA

In [14]:
cca = CCA(n_components=6)
cca.fit(new_X, new_y)
cca_X = cca.transform(new_X)
reg = LinearRegression().fit(cca_X, new_y)
np.mean((reg.predict(cca_X) - new_y)**2) ** 0.5

27780.730265985967